In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
DATA_PATH = '../data/'
LIGHTCURVES_PATH = DATA_PATH + 'lightcurves/'
FEATURES_PATH = DATA_PATH + 'features/'

In [3]:
import numpy as np
import pandas as pd
import measurements, extract
import matplotlib.pyplot as plt
import inputs2

In [4]:
def unique_ids_list(df_lcs):
    return df_lcs.index.get_level_values('ID').unique().format()

def print_num_ids_shape(df_lcs):
    unique_ids = unique_ids_list(df_lcs)
    print('Num IDs: {}  Shape: {}'.format(len(unique_ids), df_lcs.shape))

#### Import

Import __transient__ catalogue

In [5]:
df_cat = inputs2.load_transient_catalog()

Import __transient__ lightcurves

In [6]:
filename = 'transient_lightcurves_clean.pickle'
indir = LIGHTCURVES_PATH; filepath = indir + filename
df_transient_noclass = pd.read_pickle(filepath)
print_num_ids_shape(df_transient_noclass)

Num IDs: 4869  Shape: (440469, 3)


Import __non-transient__ light curves

In [7]:
filename = 'nontransient_lightcurves_clean.pickle'
indir = LIGHTCURVES_PATH; filepath = indir + filename
df_nont = pd.read_pickle(filepath)
print_num_ids_shape(df_nont)

Num IDs: 16940  Shape: (1802695, 3)


#### Add class

__Transient__

In [8]:
df_tra = df_transient_noclass.join(df_cat, how='inner')

__Non-Transient__

In [9]:
df_nont['class'] = 'non-transient'

#### Filter

In [10]:
def filter_light_curves(df_lcs, min_obs):
    df_count = df_lcs.groupby('ID', as_index=True).count()
    df_count['ObsCount'] = df_count['Mag']
    df_count = df_count[['ObsCount']]
    df_lcs_with_counts = df_lcs.join(df_count, how='inner')
    # Remove objects with less than min_obs
    df_filtered = df_lcs_with_counts[df_lcs_with_counts.ObsCount >= min_obs]
#     # Remove ObsCount
#     df_filtered = df_filtered.drop(['ObsCount'], axis=1)
    return df_filtered

def sample(df_lcs, num_samples):
    # Set random seed
    np.random.seed(42)
    # Sample non-transient subset of same size as transients
    IDs = np.random.choice(unique_ids_list(df_lcs), size=num_samples, replace=False)
#     print(IDs); return
    df_sampled = df_nont.loc[IDs]
    return df_sampled

Filter __transient__ light curves

In [11]:
df_tra_5 = filter_light_curves(df_tra, 5)
print_num_ids_shape(df_tra_5)

del df_tra

Num IDs: 4269  Shape: (438897, 5)


Filter __non-transient__ lightcurves

In [12]:
df_nont_5 = filter_light_curves(df_nont, 5)
print_num_ids_shape(df_nont_5)

Num IDs: 15193  Shape: (1798465, 5)


#### Oversample

In [13]:
def oversample(df_lcs, copies=0):
    df_oversample = df_lcs.copy()
    df_oversample['copy_num'] = 0
    for i in range(1, copies+1):
        df_temp = df_lcs.copy()
        df_temp['copy_num'] = i
        df_temp['Mag'] = np.random.normal(df_lcs.Mag, df_lcs.Magerr)
        df_oversample = df_oversample.append(df_temp)
        
    df_oversample = df_oversample.set_index(['copy_num'], append=True)
    return df_oversample

Oversample __transient__ light curves

In [14]:
df_tra_5_os = oversample(df_tra_5, 10)
print_num_ids_shape(df_tra_5_os)

del df_tra_5

Num IDs: 4269  Shape: (4827867, 5)


"Oversample" __nontransient__ light curves

In [15]:
df_nont_5_os = oversample(df_nont_5, 0)
print_num_ids_shape(df_nont_5)

del df_nont_5

Num IDs: 15193  Shape: (1798465, 5)


#### Feature Extraction

In [16]:
def extract_features(df_lcs):
    # Extract num_copy list
    num_copy_list = df_lcs.index.get_level_values('copy_num').unique()    
    num_copies = len(num_copy_list)
    
    # Extract IDs list
    unique_ids_list = df_lcs.index.get_level_values('ID').unique()
    num_ids = len(unique_ids_list)

    # Create empty feature dict
    feats_dict = extract.feature_dict(30)
    feats_dict['ObsCount'] = []
    feats_dict['Class'] = []
    # Add 'ID' and 'copy_num' index lists
    index_id_list = []
    index_copy_num_list = []
    
    num_objects = num_ids*num_copies
    for num_copy in num_copy_list:
        for i, obj_id in enumerate(unique_ids_list):
            # Print status
            current_object_i = (num_copy+1)*i
            if(current_object_i%int(num_objects/10) == 0):
                print(current_object_i, '/', num_objects)
            # Get current object light curve
            df_object = df_lcs.loc[obj_id,:,num_copy]
            # Get features
            obj_feats = extract.features(df_object, feats_dict)
            # Append features
            for k,v in obj_feats.items():
                feats_dict[k].append(obj_feats[k])
            # Append Indexes
            index_id_list.append(obj_id)
            index_copy_num_list.append(num_copy)
            # Append class and obs_count
            assert(len(df_object['class'].unique()) == 1)
            assert(len(df_object['ObsCount'].unique()) == 1)
            assert(df_object['ObsCount'].unique()[0] == df_object.shape[0])
            feats_dict['Class'].append(df_object['class'].unique()[0])
            feats_dict['ObsCount'].append(df_object.shape[0])
            
            
    # Create feature dataframe
    df_feats = pd.DataFrame(feats_dict).set_index([index_id_list,index_copy_num_list])
    df_feats.index.names = ['ID', 'copy_num']
    return df_feats

def save_features(df_feats, obj_type):
    outdir = FEATURES_PATH
    filename_raw = '{}.pickle'
    filename = filename_raw.format(obj_type)
    assert(df_feats.shape[1]==32) # 30 + ['num_obs'+'class']
    df_feats.to_pickle(outdir + filename)

#### Generate Features

In [17]:
def generate_features(df_all, transient, min_obs):
    obj_type = 'T' if transient else 'NT'
    # Generate features based on light curves
    df_feats = extract_features(df_all)
    # Save features
    save_features(df_feats, obj_type)
    # Log Finished
    print('Finished task type={} obs={}'.format(obj_type, min_obs) )
    return df_feats

Generate features __transient__ light curves

In [18]:
df_tra_feats = generate_features(df_tra_5_os, transient=True, min_obs=5)
df_nont_feats = generate_features(df_nont_5_os, transient=False, min_obs=5)

0 / 46959


/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks

0 / 46959


/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks

0 / 46959
4695 / 46959


...

Finished task type=T obs=5
0 / 15193
1519 / 15193
3038 / 15193
4557 / 15193
6076 / 15193
7595 / 15193


/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)
/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)


9114 / 15193
10633 / 15193
12152 / 15193
13671 / 15193


/Users/diego/Desktop/astro/repo/notebooks/measurements.py:220: RankWarning: Polyfit may be poorly conditioned
  p4 = np.polyfit(x, y, 4)


15190 / 15193
Finished task type=NT obs=5
